## Iremos avançar para a melhora do indicador
* Vamos adicionar um novo tratamento das colunas de texto

In [10]:
import pandas as pd

In [11]:
# Visualizando a base de treino
treino = pd.read_csv('train.csv')
treino.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [12]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


- **Fazendo o mesmo tratamento inicial que fizemos anteriormente**

In [13]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [15]:
# Usando a média para substituir valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

In [16]:
# Tratando a coluna Embarked da base de treino usando a moda 
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [17]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

### Podemos agora entender as colunas de texto
- Vamos agora **adicionar um novo tratamento das colunas de texto**

In [18]:
# Verificando as colunas de texto na base de treino
treino.columns[treino.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [19]:
# Verificando os valores na coluna Sex
treino.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [20]:
# e na coluna Embarked
treino.Embarked.value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

- Para tratar a coluna Sex, podemos criar uma nova coluna chamada **"MaleCheck"** que **vai receber 1 se o gênero for masculino e 0 se o gênero for feminino**

In [21]:
# Usando uma lambda function para fazer esse tratamento
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [22]:
# Verificando os valores
treino[['Sex','MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
Name: count, dtype: int64

- O próximo passo é fazer o mesmo para a base de teste

In [23]:
# Usando uma lambda function para fazer esse tratamento
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [24]:
# Verificando os valores
teste[['Sex','MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
Name: count, dtype: int64

- Agora, para tratar a coluna **Embarked** usando o **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** que irá **criar uma nova coluna para cada um dos rótulos da coluna original**

In [25]:
# Importando o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [26]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [27]:
# Fazendo o fit com os dados
ohe = ohe.fit(treino[['Embarked']])

In [28]:
# Fazendo a transformação
ohe.transform(treino[['Embarked']]).toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

In [29]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(), columns=ohe.get_feature_names_out())
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1


In [30]:
# Podemos agora adicionar essa coluna na nossa base de treino
treino = pd.concat([treino, ohe_df], axis=1)

In [31]:
# Verificando os valores
treino[['Embarked_C','Embarked_Q','Embarked_S','Embarked']].value_counts()

Embarked_C  Embarked_Q  Embarked_S  Embarked
0           0           1           S           646
1           0           0           C           168
0           1           0           Q            77
Name: count, dtype: int64

- Agora vamos **fazer o mesmo para a base de teste usando o encoder ohe que criamos acima**

In [32]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(teste[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())

In [33]:
# Adicionando o resultado na base de teste
teste = pd.concat([teste,ohe_df],axis=1)

In [34]:
# Verificando também os valores
teste[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
Name: count, dtype: int64

### Usando essa nova base no modelo

In [35]:
# Visualizando a base
treino.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1


In [36]:
# Podemos então apagar essas duas colunas que já tratamos
treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

- Podemos selecionar os mesmos modelos que vimos anteriormente
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [37]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [38]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

In [39]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

In [40]:
# Fazendo a importação
from sklearn import tree
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [41]:
# Importando
from sklearn.neighbors import KNeighborsClassifier
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [42]:
# Importando
from sklearn.linear_model import LogisticRegression
# Criando o classificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [43]:
# Importando
from sklearn.metrics import accuracy_score

In [45]:
# Para a árvore de decisão
accuracy_score(y_val, y_pred_ac)
# Para K-NN
accuracy_score(y_val, y_pred_knn)
# Para Regressão Logística
accuracy_score(y_val, y_pred_rl)

print(f"Acurácia para a Decision Tree: {round(accuracy_score(y_val, y_pred_ac),3)}")
print(f"Acurácia para a K-NN: {round(accuracy_score(y_val, y_pred_knn),3)}")
print(f"Acurácia para a Regressão Logística: {round(accuracy_score(y_val, y_pred_rl),3)}")

Acurácia para a Decision Tree: 0.749
Acurácia para a K-NN: 0.715
Acurácia para a Regressão Logística: 0.817


* Perceba que aumentamos a acurácia para de 0.725 para 0.817 com a adição das features Sex e Embarked. Isso é ótimo!

- Avaliando a **matriz de confusão**

In [46]:
# Importando
from sklearn.metrics import confusion_matrix

In [47]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[138,  37],
       [ 37,  83]], dtype=int64)

In [48]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[147,  28],
       [ 56,  64]], dtype=int64)

In [49]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[153,  22],
       [ 32,  88]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [50]:
# Visualizando o X_train
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
6,1,54.000000,0,0,51.8625,1,0,0,1
718,3,29.699118,0,0,15.5000,1,0,1,0
685,2,25.000000,1,2,41.5792,1,1,0,0


In [51]:
# Visualizando a base de teste
teste.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0


In [52]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste.drop('PassengerId',axis=1)
X_teste

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,3,34.50000,0,0,7.8292,1,0,1,0
1,3,47.00000,1,0,7.0000,0,0,0,1
2,2,62.00000,0,0,9.6875,1,0,1,0
3,3,27.00000,0,0,8.6625,1,0,0,1
4,3,22.00000,1,1,12.2875,0,0,0,1
...,...,...,...,...,...,...,...,...,...
413,3,30.27259,0,0,8.0500,1,0,0,1
414,1,39.00000,0,0,108.9000,0,1,0,0
415,3,38.50000,0,0,7.2500,1,0,0,1
416,3,30.27259,0,0,8.0500,1,0,0,1


In [53]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)
# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred
# Selecionando apenas a coluna de Id e Survived para fazer o envio
teste[['PassengerId','Survived']]

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [55]:
# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred

In [56]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId','Survived']]

In [58]:
# Exportando para um csv
base_envio.to_csv('resultados2.csv',index=False)

## Conseguimos, com a manipulação e adição das features "sex" e "embarked" a pontuação de 0.76555 na competição Kaggle.
![resultados2](resultados2.png)